In [10]:
#Python specific packages
import pandas as pd
import numpy as np
import sys

In [11]:
#specifying local system path for remaining functions files
sys.path.insert(1, '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts')


In [55]:
#self written functions and scripts that will be needed
import text_normalization_funs as TN

In [56]:
import similiarity_funcs as SIM

In [14]:
#Loading in required raw data for processing, dropping all rows that contain NAs
newsJan20 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_20_news.csv').dropna(subset=['title', 'maintext'])
newsJan21 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_21_news.csv').dropna(subset=['title', 'maintext'])
newsJan23 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_23_news.csv').dropna(subset=['title', 'maintext'])
newsJan24 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_24_news.csv').dropna(subset=['title', 'maintext'])
newsJan26 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_26_news.csv').dropna(subset=['title', 'maintext'])
newsJan27 = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/raw/Jan_27_news.csv').dropna(subset=['title', 'maintext'])

In [34]:
#Concatinating all dataframes
combine_news = pd.concat([newsJan20, newsJan21, newsJan23, newsJan24, newsJan26, newsJan27], axis=0)

In [38]:
#Likely some duplicate stories references over different days and different landing pages
#Removing said duplicates
unique_news = combine_news.drop_duplicates(subset = 'mainurl')
len(unique_news)

1221

In [39]:
#Now need to isolate the similar and related links, store those in a vector and isolate those stories to fetch their inofrmaiton
print(unique_news['simlinks'])

0      https://www.cbc.ca/news/canada/prince-edward-i...
1      https://www.cbc.ca/news/canada/new-brunswick/m...
2      https://www.cbc.ca/news/canada/north/stanton-h...
3      https://www.cbc.ca/news/world/royal-romance-pr...
4                                                     []
                             ...                        
439    https://www.cbc.ca/news/world/gordon-sondland-...
441                                                   []
445    https://www.cbc.ca/news/canada/montreal/medica...
447                                                   []
457    https://www.cbc.ca/news/canada/ottawa/sin-scam...
Name: simlinks, Length: 1221, dtype: object


In [77]:
splitlinks = unique_news['simlinks'].str.split(' ', expand=True)
splitlinks_reduced = splitlinks.loc[:,1:4]
splitlinks_foradding = splitlinks_reduced.rename(columns={1:"sim1", 2:'sim2', 3:'sim3', 4:'sim4'})


In [5]:
#Adding subsetted links to main dataframe
UniqLinkNews = pd.concat([unique_news, splitlinks_foradding], axis=1)

#Export as a newdataframe
UniqLinkNews.to_csv('Cleaned_Split_RawNews.csv')
#This dataframe will be pulled into another scrip to scrape article urls.

NameError: name 'unique_news' is not defined

In [2]:
#Reading in data frame with scraped article
sim_news = pd.read_csv("/Users/Cylita/Desktop/insight-ds-project_news-link/data/processed/News_Raw_ValData.csv").dropna(subset=['title', 'maintext'])
trimmed_sim = sim_news[['author', 'date','title','maintext', 'mainurl']]

In [6]:
UniqLinkNews = pd.read_csv('/Users/Cylita/Desktop/insight-ds-project_news-link/data/processed/Cleaned_Split_RawNews.csv')
trimmed_full = UniqLinkNews[['author','date', 'title', 'maintext', 'mainurl', 'simlinks', 'sim1']]

In [16]:
#Combining the above two dataframes
raw_val = pd.concat([trimmed_full, trimmed_sim], axis=0, ignore_index = True, sort=True)

In [17]:
#For the above combined dataframe - creating combined text and cleaning 
#Combing the news story headlines with the maintext for the stories
full_text = raw_val["title"].map(str)+ '. ' + raw_val["maintext"]

#Preprocessing all text to conduct tl-dlf cosine similarity analysis
normfull = TN.normalize_NewsText(full_text)

KeyboardInterrupt: 

In [146]:
#Adding in cleaned text to dataframe
raw_val['full_text'] = full_text
raw_val['cleaned_text'] = normfull

In [149]:
#Exporting cleaned and processed text to a corpus document
raw_val.to_csv("Val_Corpus.csv")

In [18]:
#Reading back in the new dataframe to work from
val_corpus = pd.read_csv("Val_Corpus.csv")

In [44]:
len(val_corpus)

1908

In [119]:
#Removing duplciate stories that have been included in the corpus
unique_val = val_corpus.drop_duplicates(subset=['mainurl'])
type(unique_val)

pandas.core.frame.DataFrame

In [47]:
#From the cleaned text - creating a corpus and a test story
corpus = unique_val.iloc[1:len(val_corpus)]
teststory = unique_val.iloc[0]

In [120]:
type(teststory)

pandas.core.series.Series

In [48]:
#Computing similarity scores for given dataframe
scores = SIM.SimCorpCreate (corpora = corpus['cleaned_text'], directory= '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts/', querydoc = teststory['cleaned_text'])

In [49]:
test = SIM.StoryMatch_Val(SimScores = scores, originalDF = corpus, NumLinks = len(corpus))

/Users/Cylita/Desktop/insight-ds-project_news-link/scripts/similiarity_funcs.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  originalDF['sim_scores'] = SimScores


In [102]:
test.head(15)

,title,mainurl,sim1,sim_scores
344,Flurries expected for Monday morning on P.E.I.,https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.336665
100,"With snowy, windy conditions, RCMP advise Isla...",https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.323564
1380,"Snow, strong winds and bitterly cold temperatu...",https://www.cbc.ca/news/canada/nova-scotia/sno...,NaN,0.296482
299,Week should warm after bitter beginning,https://www.cbc.ca/news/canada/ottawa/ottawa-g...,NaN,0.209704
318,Poor road conditions cause school closures in ...,https://www.cbc.ca/news/canada/new-brunswick/w...,NaN,0.206494
209,Troops arrive in N.L. for blizzard cleanup,https://www.cbc.ca/news/canada/newfoundland-la...,https://www.cbc.ca/news/canada/newfoundland-la...,0.183312
1089,Parking ban in effect as city continues to cle...,https://www.cbc.ca/news/canada/ottawa/parking-...,NaN,0.171545
256,State of emergency declared in St. John's regi...,https://www.cbc.ca/news/canada/newfoundland-la...,NaN,0.168950
224,Saskatchewan cold snap ending as temperatures ...,https://www.cbc.ca/news/canada/saskatoon/envir...,https://www.cbc.ca/news/canada/saskatoon/indoo...,0.168095
273,"Slippery roads to blame for cruiser crash, pol...",https://www.cbc.ca/news/canada/ottawa/gatineau...,NaN,0.160864


In [ ]:
my1score = []
jourscore = []
top10orNaw = []


for index, row in df.iterrows():
    print(row['c1'], row['c2'])
for story in len(unique_val):
    if str(story['sim1']) != 'nan':
    
    
    else:
    

In [128]:
#Creating a dataframe that has all of the rows for analysis separated
#Dataframe keeps the original indicies for rows
news_validation = unique_val[~pd.isna(unique_val['sim1'])]

#Setting up three empty lists to store similarity scores
my1score = []
jourscore = []
top10orNaw = []

#looping the following over ever row in my data frame
for index, row in news_validation.iterrows():
    #Using the row index to separate out test story and remaining stories for text corpus
    teststory = news_validation.loc[index,]
    corpus = unique_val.drop([index], axis=0)
    
    #Computing similarity scores for given dataframe
    scores = SIM.SimCorpCreate (corpora = corpus['cleaned_text'], 
                                    directory= '/Users/Cylita/Desktop/insight-ds-project_news-link/scripts/', 
                                    querydoc = teststory['cleaned_text'])
        
    #Appending similarity scores to origina l
    valscores = SIM.StoryMatch_Val(SimScores = scores, originalDF = corpus, NumLinks = len(corpus))
    
    #Adding value of most similar story to the list
    my1score.append(valscores['sim_scores'].iloc[0])
    
    #Using function to determined if similar link is in my top 10
    top = Top10(TestScores = valscores, TestStory = teststory)
    top10orNaw.append(top)
    
    #Finding similarity score for the link directly inserted by the journalists
    journal = FindSimScore(TestScores = valscores, TestStory= teststory)
    jourscore.append(journal)


KeyboardInterrupt: 

In [125]:
test.head(5)

,title,mainurl,sim1,sim_scores
344,Flurries expected for Monday morning on P.E.I.,https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.336665
100,"With snowy, windy conditions, RCMP advise Isla...",https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.323564
1380,"Snow, strong winds and bitterly cold temperatu...",https://www.cbc.ca/news/canada/nova-scotia/sno...,NaN,0.296482
299,Week should warm after bitter beginning,https://www.cbc.ca/news/canada/ottawa/ottawa-g...,NaN,0.209704
318,Poor road conditions cause school closures in ...,https://www.cbc.ca/news/canada/new-brunswick/w...,NaN,0.206494


In [89]:
def Top10(TestScores, TestStory):
    Top10 = TestScores.iloc[0:9]
    InTen = Top10['sim1'].isin(pd.Series(TestStory['sim1']))
    
    if True in InTen:
        output = 1
    else:
        output = 0
    
    return output

In [146]:
def FindSimScore (TestScores, TestStory):
    findsim = TestScores.loc[TestScores['mainurl'] == TestStory['sim1']]
    output = findsim['sim_scores'].iloc[0]
    return output

In [147]:
b = FindSimScore(TestScores = test, TestStory = teststory)

In [148]:
print(b)

0.01119490247219801


In [100]:
b= test.loc[test['mainurl'] == teststory['sim1']]

In [ ]:
b

In [90]:
zz = Top10(TestScores = test, TestStory = teststory)

In [91]:
print (zz)

0


In [50]:
test

,title,mainurl,sim1,sim_scores
344,Flurries expected for Monday morning on P.E.I.,https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.336665
100,"With snowy, windy conditions, RCMP advise Isla...",https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,0.323564
1380,"Snow, strong winds and bitterly cold temperatu...",https://www.cbc.ca/news/canada/nova-scotia/sno...,NaN,0.296482
299,Week should warm after bitter beginning,https://www.cbc.ca/news/canada/ottawa/ottawa-g...,NaN,0.209704
318,Poor road conditions cause school closures in ...,https://www.cbc.ca/news/canada/new-brunswick/w...,NaN,0.206494
...,...,...,...,...
229,Man killed in snowmobile crash,https://www.cbc.ca/news/canada/ottawa/south-al...,NaN,0.000560
1665,Sprawling child porn investigation in southwes...,https://www.cbc.ca/news/canada/kitchener-water...,NaN,0.000537
1657,1 year after Khashoggi disappeared at Saudi Co...,https://www.cbc.ca/news/world/khashoggi-saudi-...,NaN,0.000481
657,Ontario Securities Commission chair and CEO Ma...,https://www.cbc.ca/news/business/osc-maureen-j...,NaN,0.000379


In [23]:
print(teststory)

Unnamed: 0                                                      0
author                                                  Anonymous
date                                     2020-01-20T18:54:01.116Z
maintext        Following Sunday's snowfall, crews continue to...
mainurl         https://www.cbc.ca/news/canada/prince-edward-i...
sim1            https://www.cbc.ca/news/canada/prince-edward-i...
simlinks        https://www.cbc.ca/news/canada/prince-edward-i...
title           Snow all but over ahead of a bitterly cold Tue...
full_text       Snow all but over ahead of a bitterly cold Tue...
cleaned_text    snow ahead bitterly cold tuesday pei follow su...
Name: 0, dtype: object


In [51]:
b= test.loc[test['mainurl'] == teststory['sim1']]

In [52]:
print(b)

                                                 title  \
319  How 'absolute terror' led this P.E.I. man to s...   

                                               mainurl  \
319  https://www.cbc.ca/news/canada/prince-edward-i...   

                                                  sim1  sim_scores  
319  https://www.cbc.ca/news/canada/prince-edward-i...    0.022129  


In [32]:
a=test['sim1'].isin(pd.Series(teststory['sim1']))

In [78]:
str(a)

'403     False\n344     False\n100     False\n1380    False\n299     False\n318     False\n1250    False\n209     False\n256     False\nName: sim1, dtype: bool'

In [130]:
c = FindSimScore(TestStory = teststory, TestScores = test)

In [143]:
b= c.iloc[0]

In [145]:
print(b)

0.01119490247219801


In [73]:
if a == True:
    

BOOLSTORE = []
for i in c:
    if str(c) == 'False':
        value = 1
        BOOLSTORE.append(value)
    else:
        pass

In [75]:
print(c)

[False, False, False, False, False, False, False, False, False]


In [54]:
print(c)

[False, False, False, False, False, False, False, False, False]


In [ ]:
def Validatie_Sim (val_corpora):
    if 

In [153]:
if str(val_corpus['sim1']) != 'nan':
    print("Great")
else:
    print("SHOOT")

Great


In [150]:
#Now need to write a function that takes 

for i in 200:
    corpus = val_corpus.iloc[(i+1):len(val_corpus)]
    teststor = val_corpus.iloc[i]

,Unnamed: 0,author,date,maintext,mainurl,sim1,simlinks,title,full_text,cleaned_text
0,0,Anonymous,2020-01-20T18:54:01.116Z,"Following Sunday's snowfall, crews continue to...",https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,https://www.cbc.ca/news/canada/prince-edward-i...,Snow all but over ahead of a bitterly cold Tue...,Snow all but over ahead of a bitterly cold Tue...,snow ahead bitterly cold tuesday pei follow su...
1,1,Anonymous,2020-01-17T21:53:16.952Z,A biologist teaching at the Maritime College o...,https://www.cbc.ca/news/canada/new-brunswick/r...,https://www.cbc.ca/news/canada/new-brunswick/d...,https://www.cbc.ca/news/canada/new-brunswick/m...,Forestry professor was fired for harassment an...,Forestry professor was fired for harassment an...,forestry professor fire harassment discriminat...
2,2,Anonymous,2019-12-09T17:43:42.150Z,An Alaska dentist accused of fraud and unneces...,https://www.cbc.ca/news/canada/north/alaska-de...,NaN,https://www.cbc.ca/news/canada/north/stanton-h...,"Prosecutors say Alaska dentist, accused of fra...","Prosecutors say Alaska dentist, accused of fra...",prosecutor say alaska dentist accuse fraud rod...
3,3,Zulekha Nathoo,2020-01-18T23:09:29.618Z,"Prince Harry and Meghan, the Duke and Duchess ...",https://www.cbc.ca/news/world/harry-meghan-los...,https://www.cbc.ca/news/world/harry-meghan-ste...,https://www.cbc.ca/news/world/royal-romance-pr...,Prince Harry and Meghan will give up royal tit...,Prince Harry and Meghan will give up royal tit...,prince harry meghan give royal title public fu...
4,4,Rosa Marchitelli,2019-04-15T16:17:03.386Z,Rania Khunaysir didn't see it coming when she ...,https://www.cbc.ca/news/health/airline-cabin-p...,NaN,[],Pesticide on planes: Parents call for changes ...,Pesticide on planes: Parents call for changes ...,pesticide plane parent call change cabin fumig...
5,5,Anonymous,2020-01-16T22:06:41.621Z,Shares in Bombardier Inc. plunged more than 30...,https://www.cbc.ca/news/business/bombardier-do...,https://www.cbc.ca/news/business/bombardier-bu...,https://www.cbc.ca/news/business/bombardier-hi...,Bombardier shares drop after it says revenue w...,Bombardier shares drop after it says revenue w...,bombardi share drop say revenue fall short sha...
6,6,Anonymous,2020-01-21T00:13:16.841Z,Suncor Energy has been ordered to immediately ...,https://www.cbc.ca/news/canada/newfoundland-la...,NaN,[],Terra Nova oil platform ordered to stop all wo...,Terra Nova oil platform ordered to stop all wo...,terra nova oil platform order stop work confin...
7,7,Anonymous,2020-01-15T20:13:53.817Z,The decade that just ended was by far the hott...,https://www.cbc.ca/news/technology/hottest-dec...,https://www.cbc.ca/news/technology/what-on-ear...,https://www.cbc.ca/news/technology/global-temp...,Earth just had its hottest decade on record,Earth just had its hottest decade on record. T...,earth hot decade record decade end far hot eve...
8,8,Erica Johnson,2019-04-02T17:51:22.864Z,Matthew Tomlinson spent five months trying to ...,https://www.cbc.ca/news/business/telecom-telev...,NaN,[],Bell customer's 5-month fight is 1 of almost 1...,Bell customer's 5-month fight is 1 of almost 1...,bell customer month fight almost recent compla...
9,9,Anonymous,2020-01-20T15:14:11.237Z,Malaysia has sent back 150 containers of plast...,https://www.cbc.ca/news/world/malaysia-sends-b...,https://www.cbc.ca/news/world/we-are-going-to-...,https://www.cbc.ca/news/world/sp-malaysia-cana...,"Malaysia sends back 150 containers of trash, s...","Malaysia sends back 150 containers of trash, s...",malaysia send back container trash say world r...
